# Obtener los datos del Parkinson

**APIS**

In [93]:
import pandas as pd
import requests

# URLs de la API
DATA_URL = "https://api.ourworldindata.org/v1/indicators/916408.data.json"
METADATA_URL = "https://api.ourworldindata.org/v1/indicators/916408.metadata.json"


**CARGA DE DATOS**

In [94]:
def cargar_datos():
   
    datos = requests.get(DATA_URL).json()  # Datos de casos de Parkinson
    metadata = requests.get(METADATA_URL).json()  # Información adicional (años y países)
    return datos, metadata


**PROCESAR DATOS**

In [95]:
def procesar_datos(datos, metadata):
    """Extrae la lista de valores, años disponibles y países desde los datos JSON."""
    
    # Extraer la lista de valores numéricos (casos de Parkinson)
    valores = datos["values"]
    
    # Extraer la lista de años disponibles
    años = []
    for item in metadata["dimensions"]["years"]["values"]:
        años.append(item["id"])
    
    # Extraer los países y sus nombres en un diccionario
    paises = {}
    for item in metadata["dimensions"]["entities"]["values"]:
        paises[item["id"]] = item["name"]
    
    return valores, años, paises


**CREAR EL DATAFRAME**

In [96]:
def crear_dataframe(valores, años, paises):
    
    num_años = len(años)
    num_valores = len(valores)

    if num_valores % num_años == 0:
        num_paises = num_valores // num_años
        

        # Tabla con datos
        datos_lista = []
        for i in range(num_valores):
            año = años[i % num_años]  # Seleccionar el año
            id_pais = list(paises.keys())[i // num_años]  # Seleccionar el país 
            nombre_pais = paises[id_pais]  #  nombre del país

            # Crear un diccionario con los datos y añadir a la lista
            fila = {"Año": año, "País": nombre_pais, "Parkisnon": valores[i]}
            datos_lista.append(fila)

        # Convertir la lista de datos en un DataFrame
        df = pd.DataFrame(datos_lista)

    else:
       
        # Crear un DataFrame con los datos disponibles
        df = pd.DataFrame({"Año": años[:num_valores], "Casos": valores[:num_valores]})
        df["País"] = "Mundo"

    return df

In [97]:
datos, metadata = cargar_datos()  
valores, años, paises = procesar_datos(datos, metadata) 
df_parkinson = crear_dataframe(valores, años, paises) 

# Mostrar los resultados en un display
display(df_parkinson)

,Año,País,Parkisnon
0,1990,Cambodia,21.533830
1,1991,Cambodia,21.483840
2,1992,Cambodia,21.502138
3,1993,Cambodia,21.607151
4,1994,Cambodia,21.767658
...,...,...,...
7291,2017,South America,101.905710
7292,2018,South America,105.467110
7293,2019,South America,108.644905
7294,2020,South America,110.542580


# Obtener los datos de tasa de contaminación de aire

**PROCESAR DATOS**

In [117]:
def procesar_datos(datos, metadata):
    # Extraer valores y años
    valores = datos['values']
    años = [item['id'] for item in metadata['dimensions']['years']['values']]

    # Extraer información de entidades
    entidades = metadata['dimensions']['entities']['values']
    nombres_entidades = [entidad['name'] for entidad in entidades]
    

    # Ajustar la longitud de los valores si no coincide con las entidades
    if len(valores) != len(nombres_entidades) * len(años):
        print("Las longitudes de los datos no coinciden. Verifica los datos.")
        return None

    # Crear listas para el DataFrame
    años_repetidos = años * len(nombres_entidades)
    entidades_repetidas = sum([[nombre] * len(años) for nombre in nombres_entidades], [])
    

    # Crear un diccionario con los datos procesados
    datos_procesados = {
        'Año': años_repetidos,
        'País': entidades_repetidas,
        'Tasa_contaminacion_Aire': valores
    }

    return datos_procesados

**CREAR DATAFRAME**

In [118]:
def crear_dataframe(datos_procesados):
    if datos_procesados:
        df = pd.DataFrame(datos_procesados)
        return df
    else:
        print("No se pudo crear el DataFrame. Los datos procesados son nulos.")
        return None

In [119]:
import pandas as pd
import requests
import json

# URLs de la API
DATA_URL = "https://api.ourworldindata.org/v1/indicators/939832.data.json"
METADATA_URL = "https://api.ourworldindata.org/v1/indicators/939832.metadata.json"

datos, metadata = cargar_datos()
datos_procesados = procesar_datos(datos, metadata)
df_aire = crear_dataframe(datos_procesados)
display(df_aire)

,Año,País,Tasa_contaminacion_Aire
0,1990,Afghanistan,9.830483
1,1991,Afghanistan,9.412167
2,1992,Afghanistan,8.970597
3,1993,Afghanistan,8.853374
4,1994,Afghanistan,9.138245
...,...,...,...
7099,2017,Zimbabwe,2.167512
7100,2018,Zimbabwe,2.128228
7101,2019,Zimbabwe,2.164050
7102,2020,Zimbabwe,2.195074


# Obtener los datos de la variable de calidad de agua (nitratos)

**PROCESAR DATOS**

In [120]:
def procesar_datos(data_response, metadata_response):
    
    # Lista de valores de concentración de nitratos
    valores = data_response["values"]
    
    # Lista de años disponibles
    años = []
    for item in metadata_response["dimensions"]["years"]["values"]:
        años.append(item["id"])
    
    # Diccionario de países con su código y nombre
    paises = {}
    for item in metadata_response["dimensions"]["entities"]["values"]:
        paises[item["id"]] = item["name"]
    
    return valores, años, paises

**CREAR DATAFRAME**

In [121]:
def crear_dataframe(valores, años, paises):
    
    datos_lista = []
    indice_valor = 0  

    for id_pais, nombre_pais in paises.items():  # Iterar sobre los países
        for año in años:  # Iterar sobre los años disponibles
            
            if indice_valor < len(valores):
                # Añadir el valor si hay datos disponibles
                fila = {
                    "Año": año,
                    "País": nombre_pais,                    
                    "Nitratos": valores[indice_valor]
                }
                indice_valor += 1  
            
            else:
                # Si no hay más valores, completar con None 
                fila = {
                    "Año": año,
                    "País": nombre_pais,                    
                    "Nitratos": None
                }
            
            datos_lista.append(fila)  # Añadir la fila a la lista de datos
    
    # Convertir la lista en un DataFrame 
    df = pd.DataFrame(datos_lista)
    
    return df

In [122]:
import pandas as pd
import requests
DATA_URL = "https://api.ourworldindata.org/v1/indicators/820661.data.json"
METADATA_URL = "https://api.ourworldindata.org/v1/indicators/820661.metadata.json"
datos, metadata = cargar_datos()  
valores, años, paises = procesar_datos(datos, metadata) 
df_nitratos = crear_dataframe(valores, años, paises) 

# Mostrar los resultados en un display
display(df_nitratos)

,Año,País,Nitratos
0,1992,Austria,27.586168
1,1993,Austria,26.506536
2,1994,Austria,26.761826
3,1995,Austria,29.362993
4,1996,Austria,30.037287
...,...,...,...
1105,2017,Turkey,NaN
1106,2018,Turkey,NaN
1107,2019,Turkey,NaN
1108,2020,Turkey,NaN


# Obtener los datos de la variable de exposición al plomo

**PRCESAR DATOS**

In [123]:
def procesar_metadata(metadata_response, data_response):
    # Crear un diccionario vacío 
    entities_map = {}
    
    # Iterar sobre las entidades en metadata 
    for e in metadata_response["dimensions"]["entities"]["values"]:
        entity_id = e["id"]
        entity_name = e["name"]
        entities_map[entity_id] = entity_name

    # Verificar que las listas de 'values', 'years' y 'entities' tengan la misma longitud
    if not (len(data_response["values"]) == len(data_response["years"]) == len(data_response["entities"])):
        print("Las listas 'values', 'years' y 'entities' tienen diferentes longitudes.")
    
    return entities_map

**CREAR DATAFRAME**

In [124]:
def crear_dataframe(data_response, entities_map):
    # Crear una lista vacía para almacenar las filas de datos
    data_list = []
    
    # Iterar sobre los valores de 'values', 'years' y 'entities' para construir el DataFrame
    for i in range(len(data_response["values"])):
        value = data_response["values"][i]
        year_id = data_response["years"][i]
        entity_id = data_response["entities"][i]
        
        # Obtener el nombre de la entidad
        entity_name = entities_map.get(entity_id, "Unknown")
        
        # Añadir los datos a la lista
        data_list.append({"Año": year_id,"País": entity_name, "Exp_Plomo": value})

    # Convertir la lista a un DataFrame
    df = pd.DataFrame(data_list)
    
    return df

In [125]:
import pandas as pd
import requests
DATA_URL = "https://api.ourworldindata.org/v1/indicators/941463.data.json"
METADATA_URL = "https://api.ourworldindata.org/v1/indicators/941463.metadata.json"
data_response, metadata_response = cargar_datos()  
entities_map = procesar_metadata(metadata_response, data_response)  
df_plomo = crear_dataframe(data_response, entities_map)  

# Mostrar las filas del DataFrame
display(df_plomo)

,Año,País,Exp_Plomo
0,1990,Afghanistan,2765.59400
1,1991,Afghanistan,2773.61840
2,1992,Afghanistan,2782.52610
3,1993,Afghanistan,2818.00600
4,1994,Afghanistan,2869.15920
...,...,...,...
7099,2017,Zimbabwe,759.63760
7100,2018,Zimbabwe,755.67847
7101,2019,Zimbabwe,749.21820
7102,2020,Zimbabwe,739.96454


# Obtener los datos de uso de pepticidas

**PROCESAR DATOS**

In [126]:
def procesar_datos(data_response, metadata_response):
    
    # Lista de valores de pesticidas
    valores = data_response["values"]
    
    # Lista de años disponibles
    años = []
    for item in metadata_response["dimensions"]["years"]["values"]:
        años.append(item["id"])
    
    # Diccionario de países con su código y nombre
    paises = {}
    for item in metadata_response["dimensions"]["entities"]["values"]:
        paises[item["id"]] = item["name"]
    
    return valores, años, paises

**CREAR DATAFRAME**

In [127]:
def crear_dataframe(valores, años, paises):
    """Crea un DataFrame a partir de los datos procesados."""
    
    datos_lista = []
    indice_valor = 0  

    for id_pais, nombre_pais in paises.items():  # Iterar sobre los países
        for año in años:  # Iterar sobre los años disponibles
            
            if indice_valor < len(valores):
                # Añadir el valor si hay datos disponibles
                fila = {
                    "Año": año,
                    "País": nombre_pais,
                    "Pesticidas": valores[indice_valor]
                }
                indice_valor += 1  
            
            else:
                # Si no hay más valores, completar con None 
                fila = {
                    "Año": año,
                    "País": nombre_pais,
                    "Pesticidas": None
                }
            
            datos_lista.append(fila)  # Añadir la fila a la lista de datos
    
    # Convertir la lista en un DataFrame
    df = pd.DataFrame(datos_lista)
    
    return df

In [128]:
import pandas as pd
import requests
DATA_URL = "https://api.ourworldindata.org/v1/indicators/832404.data.json"
METADATA_URL = "https://api.ourworldindata.org/v1/indicators/832404.metadata.json"
datos, metadata = cargar_datos()  
valores, años, paises = procesar_datos(datos, metadata) 
df_pepticidas = crear_dataframe(valores, años, paises) 


# Mostrar las filas del DataFrame
display(df_pepticidas)

,Año,País,Pesticidas
0,1990,Africa (FAO),73958.66
1,1991,Africa (FAO),69956.26
2,1992,Africa (FAO),60260.71
3,1993,Africa (FAO),54301.62
4,1994,Africa (FAO),58819.04
...,...,...,...
8027,2017,Zimbabwe,NaN
8028,2018,Zimbabwe,NaN
8029,2019,Zimbabwe,NaN
8030,2020,Zimbabwe,NaN


# UNION DE DATOS

## Teniendo en cuenta todos los datos de todos los DF

In [129]:
df_merged_outer = df_parkinson.merge(df_nitratos, on=['Año', 'País'], how='outer')\
                        .merge(df_plomo, on=['Año', 'País'], how='outer')


In [130]:
display(df_merged_outer)

,Año,País,Parkisnon,Nitratos,Exp_Plomo
0,1990,Cambodia,21.533830,NaN,799.84644
1,1991,Cambodia,21.483840,NaN,798.64690
2,1992,Cambodia,21.502138,NaN,799.89874
3,1993,Cambodia,21.607151,NaN,804.59204
4,1994,Cambodia,21.767658,NaN,812.10320
...,...,...,...,...,...
7381,2017,Liechtenstein,NaN,15.146489,NaN
7382,2018,Liechtenstein,NaN,15.881583,NaN
7383,2019,Liechtenstein,NaN,15.655723,NaN
7384,2020,Liechtenstein,NaN,20.443571,NaN


## Teniendo en cuenta todos los datos de todos los DF (+ pepticidas)

In [131]:
df_merged_outer = df_parkinson.merge(df_nitratos, on=['Año', 'País'], how='outer')\
                        .merge(df_plomo, on=['Año', 'País'], how='outer')\
                        .merge(df_pepticidas, on=['Año', 'País'], how='outer')

In [132]:
display(df_merged_outer)

,Año,País,Parkisnon,Nitratos,Exp_Plomo,Pesticidas
0,1990,Cambodia,21.533830,NaN,799.84644,258.02
1,1991,Cambodia,21.483840,NaN,798.64690,653.00
2,1992,Cambodia,21.502138,NaN,799.89874,545.00
3,1993,Cambodia,21.607151,NaN,804.59204,687.00
4,1994,Cambodia,21.767658,NaN,812.10320,2196.54
...,...,...,...,...,...,...
9269,2017,Yugoslavia,NaN,NaN,NaN,NaN
9270,2018,Yugoslavia,NaN,NaN,NaN,NaN
9271,2019,Yugoslavia,NaN,NaN,NaN,NaN
9272,2020,Yugoslavia,NaN,NaN,NaN,NaN


## Teniendo en cuenta los datos comunes de todos los DF

In [133]:
df_merged_inner = df_parkinson.merge(df_nitratos, on=['Año', 'País'], how='inner')\
                              .merge(df_plomo, on=['Año', 'País'], how='inner')

In [134]:
display(df_merged_inner)

,Año,País,Parkisnon,Nitratos,Exp_Plomo
0,1992,Czechia,115.57936,8.090960,262.31787
1,1993,Czechia,117.58800,7.254836,253.40916
2,1994,Czechia,119.72034,8.145075,251.62610
3,1995,Czechia,122.08167,7.585634,249.17851
4,1996,Czechia,124.39523,7.122982,238.00769
...,...,...,...,...,...
1015,2017,Turkey,93.22251,NaN,326.82510
1016,2018,Turkey,96.95188,NaN,310.28693
1017,2019,Turkey,100.30327,NaN,306.72952
1018,2020,Turkey,99.63544,NaN,304.11856


## Teniendo en cuenta los datos comunes de todos los DF (+ pepticidas)

In [139]:
df_merged_inner_prueba = df_parkinson.merge(df_aire, on=['Año', 'País'], how='inner')\
                              .merge(df_plomo, on=['Año', 'País'], how='inner')\
                              .merge(df_pepticidas, on=['Año', 'País'], how='inner')\
                              .merge(df_nitratos, on=['Año','País'],how = 'inner')

In [140]:
display(df_merged_inner_prueba)

,Año,País,Parkisnon,Tasa_contaminacion_Aire,Exp_Plomo,Pesticidas,Nitratos
0,1992,Czechia,115.57936,1.455477,262.31787,1105.73,8.090960
1,1993,Czechia,117.58800,1.443997,253.40916,1105.73,7.254836
2,1994,Czechia,119.72034,1.298633,251.62610,1105.73,8.145075
3,1995,Czechia,122.08167,1.113701,249.17851,1105.73,7.585634
4,1996,Czechia,124.39523,0.912675,238.00769,1105.73,7.122982
...,...,...,...,...,...,...,...
1015,2017,Turkey,93.22251,5.153062,326.82510,84.03,NaN
1016,2018,Turkey,96.95188,4.861793,310.28693,88.60,NaN
1017,2019,Turkey,100.30327,4.874065,306.72952,78.25,NaN
1018,2020,Turkey,99.63544,4.889884,304.11856,71.71,NaN


In [145]:
# Después de hacer el merge, puedes eliminar las filas con NaN en 'Nitratos'
df_merged_final =df_merged_inner_prueba.dropna(subset=['Nitratos'])

# Verifica los resultados
display(df_merged_final)


,Año,País,Parkisnon,Tasa_contaminacion_Aire,Exp_Plomo,Pesticidas,Nitratos
0,1992,Czechia,115.57936,1.455477,262.317870,1105.73,8.090960
1,1993,Czechia,117.58800,1.443997,253.409160,1105.73,7.254836
2,1994,Czechia,119.72034,1.298633,251.626100,1105.73,8.145075
3,1995,Czechia,122.08167,1.113701,249.178510,1105.73,7.585634
4,1996,Czechia,124.39523,0.912675,238.007690,1105.73,7.122982
...,...,...,...,...,...,...,...
805,2017,Iceland,273.64038,0.527082,75.714800,828.62,4.273996
806,2018,Iceland,278.41790,0.755430,72.021164,873.01,4.726720
807,2019,Iceland,283.65630,0.699459,69.151436,824.14,4.639593
808,2020,Iceland,286.52228,0.635466,64.987210,1062.61,5.769000
